## Timing Comparisons
This notebook compares timings for fetching short random genomic slices from GRCh38 using SeqRepo and NCBI E-Utilities. (Code for RefGet is provided below, but no collection with human reference data is currently available.)

In [1]:
from base64 import urlsafe_b64decode, urlsafe_b64encode
from binascii import hexlify, unhexlify
import hashlib
import itertools
import os
import random
import time

import requests

from biocommons.seqrepo import SeqRepo

In [2]:
# setup fetch_seqrepo(...)

sr = SeqRepo("/usr/local/share/seqrepo/latest")

def fetch_seqrepo(accession, start=None, end=None):
    return sr.fetch(accession, start, end)

In [3]:
# setup fetch_ncbi(...)

efetch_url = "https://eutils.ncbi.nlm.nih.gov/entrez/eutils/efetch.fcgi"
rate_limit = 10                       # queries/sec
query_period = 1.0/rate_limit         # period between queries

ncbi_session = requests.Session()
ncbi_session.params.update({
    "db": "nucleotide",
    "rettype": "fasta",
    "retmode": "text",
    "api_key": os.environ["NCBI_API_KEY"],
    "tool": os.environ["NCBI_API_TOOL"],
    "email": os.environ["NCBI_API_EMAIL"]
})

from requests.adapters import HTTPAdapter
ncbi_session.mount("https://", HTTPAdapter(max_retries=3))

ncbi_last_query_time = 0
def fetch_ncbi(accession, start=None, end=None):
    global ncbi_last_query_time
    params = {"id": accession}
    if start:
        params["seq_start"] = start + 1
    if end:
        params["seq_stop"] = end
    time_now = time.time()
    sleep_time = max(0, ncbi_last_query_time + query_period - time_now)
    time.sleep(sleep_time)
    ncbi_last_query_time = time_now
    resp = ncbi_session.get(efetch_url, params=params)
    resp.raise_for_status()
    fasta = resp.text
    return fasta[fasta.find("\n")+1:].replace("\n","")

In [4]:
# setup fetch_refget_cram(...)

refget_session = requests.Session()
refget_session.params.update({
    "accept": "text/plain"
})

refget_base_url = "https://www.ebi.ac.uk/ena/cram/sequence"
    
def fetch_refget_cram(md5, start=None, end=None):
    url = refget_base_url + "/" + md5
    params = {}
    if start:
        params["start"] = start
    if end:
        params["end"] = end
    resp = refget_session.get(url, params=params)
    resp.raise_for_status()
    return resp.text    

### Build information to fetch

In [5]:
# acs: list of grch38 primary accession refseq ids
acs = [
    'NC_000001.11', 'NC_000002.12', 'NC_000003.12', 'NC_000004.12', 'NC_000005.10',
    'NC_000006.12', 'NC_000007.14', 'NC_000008.11', 'NC_000009.12', 'NC_000010.11',
    'NC_000011.10', 'NC_000012.12', 'NC_000013.11', 'NC_000014.9',  'NC_000015.10',
    'NC_000016.10', 'NC_000017.11', 'NC_000018.10', 'NC_000019.10', 'NC_000020.11',
    'NC_000021.9',  'NC_000022.11', 'NC_000023.11', 'NC_000024.10']

In [6]:
# ac_lengths = {refseq_ac: sequence length} (from SeqRepo metadata)
ac_lengths = {ac: len(sr[ac]) for ac in acs}

In [7]:
def lookup_md5(sr, ac):
    s = [a for a in sr[ac].aliases if a.startswith("MD5:")][0]
    return s[4:]

# ac_md5s = {refseq_ac: md5} (from SeqRepo aliases)
ac_md5s = {ac: lookup_md5(sr, ac) for ac in acs}

In [14]:
# build two sets of equivalent slices, one with refseq accession, the other with md5 (for refget)
def random1():
    max_size = 25
    ac = random.choice(acs)
    start = random.randint(0, ac_lengths[ac] - max_size)
    end = start + random.randint(1, 20)
    return (ac, start, end)

n_slices = 1000
refseq_slices = [random1() for _ in range(n_slices)]
md5_slices = [(ac_md5s[rs[0]], rs[1], rs[2]) for rs in refseq_slices]

### Timing

In [9]:
from requests.exceptions import RequestException

def time1sliceset(fx, slices):
    """execute fx on each s in slices, returning tuple of (elapsed time, n_exceptions)
    
    elapsed time (rather than cpu time) is used because elapsed time is a
    better proxy for user experience
    
    """
    
    def exec1(fx, s):
        try:
            return fx(*s)
        except (RequestException) as e:
            return e
    
    t0 = time.time()
    res = [exec1(fx, s) for s in slices]
    tdelta = time.time() - t0
    n_errors = len([True for r in res if isinstance(r, Exception)])
    return (tdelta, n_errors)

In [15]:
time1sliceset(fetch_ncbi, refseq_slices)

2020-09-04 07:46:39 snafu urllib3.connectionpool[395390] WARNING Retrying (Retry(total=2, connect=None, read=None, redirect=None, status=None)) after connection broken by 'ProtocolError('Connection aborted.', RemoteDisconnected('Remote end closed connection without response'))': /entrez/eutils/efetch.fcgi?db=nucleotide&rettype=fasta&retmode=text&api_key=XXX&tool=YYY&email=ZZZ&id=NC_000011.10&seq_start=112694246&seq_stop=112694261
2020-09-04 07:46:55 snafu urllib3.connectionpool[395390] WARNING Retrying (Retry(total=2, connect=None, read=None, redirect=None, status=None)) after connection broken by 'ProtocolError('Connection aborted.', RemoteDisconnected('Remote end closed connection without response'))': /entrez/eutils/efetch.fcgi?db=nucleotide&rettype=fasta&retmode=text&api_key=XXX&tool=YYY&email=ZZZ&id=NC_000024.10&seq_start=20556827&seq_stop=20556845
2020-09-04 07:46:56 snafu urllib3.connectionpool[395390] WARNING Retrying (Retry(total=2, connect=None, read=None, redirect=None, stat

2020-09-04 07:48:18 snafu urllib3.connectionpool[395390] WARNING Retrying (Retry(total=2, connect=None, read=None, redirect=None, status=None)) after connection broken by 'ProtocolError('Connection aborted.', RemoteDisconnected('Remote end closed connection without response'))': /entrez/eutils/efetch.fcgi?db=nucleotide&rettype=fasta&retmode=text&api_key=XXX&tool=YYY&email=ZZZ&id=NC_000020.11&seq_start=14353745&seq_stop=14353749
2020-09-04 07:48:18 snafu urllib3.connectionpool[395390] WARNING Retrying (Retry(total=2, connect=None, read=None, redirect=None, status=None)) after connection broken by 'ProtocolError('Connection aborted.', RemoteDisconnected('Remote end closed connection without response'))': /entrez/eutils/efetch.fcgi?db=nucleotide&rettype=fasta&retmode=text&api_key=XXX&tool=YYY&email=ZZZ&id=NC_000014.9&seq_start=51895336&seq_stop=51895353
2020-09-04 07:48:19 snafu urllib3.connectionpool[395390] WARNING Retrying (Retry(total=2, connect=None, read=None, redirect=None, status=

2020-09-04 07:49:48 snafu urllib3.connectionpool[395390] WARNING Retrying (Retry(total=2, connect=None, read=None, redirect=None, status=None)) after connection broken by 'ProtocolError('Connection aborted.', RemoteDisconnected('Remote end closed connection without response'))': /entrez/eutils/efetch.fcgi?db=nucleotide&rettype=fasta&retmode=text&api_key=XXX&tool=YYY&email=ZZZ&id=NC_000012.12&seq_start=78988546&seq_stop=78988548
2020-09-04 07:49:49 snafu urllib3.connectionpool[395390] WARNING Retrying (Retry(total=2, connect=None, read=None, redirect=None, status=None)) after connection broken by 'ProtocolError('Connection aborted.', RemoteDisconnected('Remote end closed connection without response'))': /entrez/eutils/efetch.fcgi?db=nucleotide&rettype=fasta&retmode=text&api_key=XXX&tool=YYY&email=ZZZ&id=NC_000023.11&seq_start=40408147&seq_stop=40408161
2020-09-04 07:49:50 snafu urllib3.connectionpool[395390] WARNING Retrying (Retry(total=2, connect=None, read=None, redirect=None, status

(264.1989300251007, 0)

In [16]:
time1sliceset(fetch_seqrepo, refseq_slices)

(0.39875006675720215, 0)

In [17]:
time1sliceset(fetch_refget_cram, md5_slices)

(279.4837005138397, 0)

In [23]:
# throughput
[round(1000.0/t,1) for t in (264.1989300251007, 0.39875006675720215, 279.4837005138397)]

[3.8, 2507.8, 3.6]